feature selection
----
source: https://github.com/FIIT-IAU/IAU-course/blob/main/exercises/week-06/IAU_062_feature_selection.ipynb

In [2]:
import pandas as pd
import numpy as np
import json
from joblib import dump
from sklearn.preprocessing import StandardScaler

In [3]:
# depth of this filr in the project
file_depth = '../../..'

with open(file_depth + '/config/data_1m_config.json') as config_file:
    config = json.load(config_file)

context_data_path = config['context_data']
context_data = pd.read_csv(file_depth + context_data_path)
context_data

,user_id,movie_id,day,is_weekday,season,part_of_day,holiday,user_gender,user_age,user_occupation,...,genre_film-noir,genre_for_children,genre_horror,genre_musical,genre_mystery,genre_romance,genre_sci-fi,genre_thriller,genre_war,genre_western
0,1,1193,7,0,4,4,new_years,F,1,10,...,0,0,0,0,0,0,0,0,0,0
1,1,661,7,0,4,4,new_years,F,1,10,...,0,1,0,1,0,0,0,0,0,0
2,1,914,7,0,4,4,new_years,F,1,10,...,0,0,0,1,0,1,0,0,0,0
3,1,3408,7,0,4,4,new_years,F,1,10,...,0,0,0,0,0,0,0,0,0,0
4,1,2355,7,0,4,4,no_holiday,F,1,10,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,3,1,1,4,no_holiday,M,25,6,...,0,0,0,0,0,0,0,0,0,0
1000205,6040,1094,3,1,1,4,no_holiday,M,25,6,...,0,0,0,0,0,1,0,0,1,0
1000206,6040,562,3,1,1,4,no_holiday,M,25,6,...,0,0,0,0,0,0,0,0,0,0
1000207,6040,1096,3,1,1,4,no_holiday,M,25,6,...,0,0,0,0,0,0,0,0,0,0


categorical data to numerical

In [4]:
# load holidays
with open('../holidays.json', 'r') as json_file:
    holiday_dates = json.load(json_file)

# create index mapping
holiday_mapping = {}
for holiday_name, data in holiday_dates.items():
    holiday_mapping[holiday_name] = data['index']

# add no_holiday as one of the mapping
holiday_mapping['no_holiday'] = len(holiday_mapping)
context_data['holiday_encoded'] = context_data['holiday'].map(holiday_mapping)

# create mapping for gender values ('M' and 'F')
gender_mapping = {'F': 0, 'M': 1}
context_data['user_gender_encoded'] = context_data['user_gender'].map(gender_mapping)

context_data = context_data.drop(['holiday', 'user_gender'], axis=1)
context_data.rename(columns={'holiday_encoded': 'holiday'}, inplace=True)
context_data.rename(columns={'user_gender_encoded': 'user_gender'}, inplace=True)

context_data

,user_id,movie_id,day,is_weekday,season,part_of_day,user_age,user_occupation,user_zip_code,movie_year,...,genre_horror,genre_musical,genre_mystery,genre_romance,genre_sci-fi,genre_thriller,genre_war,genre_western,holiday,user_gender
0,1,1193,7,0,4,4,1,10,48067,1975,...,0,0,0,0,0,0,0,0,1,0
1,1,661,7,0,4,4,1,10,48067,1996,...,0,1,0,0,0,0,0,0,1,0
2,1,914,7,0,4,4,1,10,48067,1964,...,0,1,0,1,0,0,0,0,1,0
3,1,3408,7,0,4,4,1,10,48067,2000,...,0,0,0,0,0,0,0,0,1,0
4,1,2355,7,0,4,4,1,10,48067,1998,...,0,0,0,0,0,0,0,0,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,3,1,1,4,25,6,11106,1989,...,0,0,0,0,0,0,0,0,9,1
1000205,6040,1094,3,1,1,4,25,6,11106,1992,...,0,0,0,1,0,0,1,0,9,1
1000206,6040,562,3,1,1,4,25,6,11106,1995,...,0,0,0,0,0,0,0,0,9,1
1000207,6040,1096,3,1,1,4,25,6,11106,1982,...,0,0,0,0,0,0,0,0,9,1


In [5]:
target_data_path = config['target_data']
target_ratings = pd.read_csv(file_depth + target_data_path)
target_ratings

,rating_id,rating
0,0,5
1,1,3
2,2,3
3,3,4
4,4,5
...,...,...
1000204,1000204,1
1000205,1000205,5
1000206,1000206,5
1000207,1000207,4


-----------

In [6]:
# data scaling was needed
X = StandardScaler().fit_transform(context_data) 
y = target_ratings['rating']

#### Recursive feature elimination (RFE)
takes too long to run (after 5000 minutes I stopped it)

```python

In [7]:
# from sklearn.feature_selection import RFE
# from sklearn.svm import SVR

# # 15
# n_features = int(context_data.shape[1]/2)

# svr_estimator = SVR(kernel="linear")
# rfe_selector = RFE(svr_estimator, n_features_to_select=n_features, step=1)
# rfe_selector = rfe_selector.fit(X, y)

# print(rfe_selector.support_)
# print(rfe_selector.ranking_)
# print(rfe_selector.feature_names_in_)

In [8]:
## save the data

# rfe_selector_list = []
# rfe_selector_list.append(rfe_selector.support_)
# rfe_selector_list.append(rfe_selector.ranking_)
# rfe_selector_list.append(rfe_selector.feature_names_in_)

# import csv
# with open("rfe_selector_nparray.csv", "w") as f:
#     wr = csv.writer(f)
#     wr.writerows(rfe_selector_list)

# with open('rfe_selector_nparray.txt', 'w') as f:
#     for line in rfe_selector_list:
#         f.write(f"{line}\n")

#### Feature selection using SelectFromModel

##### Model : Logistic Regression

In [9]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

lm_selector = SelectFromModel(estimator=LogisticRegression(max_iter=1000)).fit(X, y)
# print(lm_selector.estimator_.coef_)
# print(lm_selector.threshold_)
print(lm_selector.get_support())
lm_selector_list = lm_selector.get_support()

X_new_log_reg = lm_selector.transform(X)
print(X_new_log_reg.shape)
# X_new_lm

[False  True False False False False  True False False  True False False
  True False False  True  True False  True  True  True False False False
 False  True  True False False False]
(1000209, 11)


##### Model : L1

In [10]:
from sklearn.datasets import load_iris
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
l1_selector = SelectFromModel(estimator=lsvc, prefit=True)
print(l1_selector.get_support())
l1_selector_list = l1_selector.get_support()

X_new_l1 = l1_selector.transform(X)
print(X_new_l1.shape)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
(1000209, 30)


---------------

In [11]:
lm_selected_keys = []
l1_selected_keys = []

for key, selected in zip(context_data.keys(), lm_selector_list):
    if selected:
        lm_selected_keys.append(key)

for key, selected in zip(context_data.keys(), l1_selector_list):
    if selected:
        l1_selected_keys.append(key)

print('Number of features selected using Logistic Regression: ', len(lm_selected_keys))
print(lm_selected_keys)

print()

print('Number of features selected using L1: ', len(l1_selected_keys))
print(l1_selected_keys)

Number of features selected using Logistic Regression:  11
['movie_id', 'user_age', 'movie_year', 'genre_animation', 'genre_documentary', 'genre_drama', 'genre_film-noir', 'genre_for_children', 'genre_horror', 'genre_thriller', 'genre_war']

Number of features selected using L1:  30
['user_id', 'movie_id', 'day', 'is_weekday', 'season', 'part_of_day', 'user_age', 'user_occupation', 'user_zip_code', 'movie_year', 'genre_action', 'genre_adventure', 'genre_animation', 'genre_comedy', 'genre_crime', 'genre_documentary', 'genre_drama', 'genre_fantasy', 'genre_film-noir', 'genre_for_children', 'genre_horror', 'genre_musical', 'genre_mystery', 'genre_romance', 'genre_sci-fi', 'genre_thriller', 'genre_war', 'genre_western', 'holiday', 'user_gender']


Saving the selected features to files

In [15]:
with open('feature_selections_results/lm_selected_features.txt', 'w') as file:
    for item in lm_selected_keys:
        file.write(str(item) + '\n')

with open('feature_selections_results/l1_selected_features.txt', 'w') as file:
    for item in l1_selected_keys:
        file.write(str(item) + '\n')

In [21]:
# l1_selected = []
# with open('feature_selections_results/lm_selected_features.txt', 'r') as file:
#     for line in file:
#         l1_selected.append(line.strip())